In [22]:
%load_ext autoreload
%autoreload 2
import dateutil
import json
from pyspark.sql import SQLContext, Row
from pyspark.mllib.feature import Word2Vec

import os
import sys
import inspect
import random
import datetime
from spark_util import save_rdd

currentdir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe()))
)
parentdir = os.path.dirname(currentdir)
gpparentdir = os.path.dirname(parentdir)
sys.path.insert(0, gpparentdir)

#from src.traces.trace_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
input_dir = '/user/ellery/readers/data/hashed_traces/rs3v2/year=2016/month=3/day=*/host=en.wikipedia.org'

In [24]:
def parse_dates(requests):
    clean = []
    for r in requests:
        try:
            r['t'] = datetime.datetime.strptime(r['t'], '%Y-%m-%d %H:%M:%S')
            clean.append(r)
        except:
            pass
    return clean

In [25]:
def sessionize(requests):

    sessions = []
    session = [requests[0]]
    for r in requests[1:]:
        d = r['t'] -  session[-1]['t']
        if d > datetime.timedelta(minutes=30):
            sessions.append(session)
            session = [r,]
        else:
            session.append(r)

    sessions.append(session)
    return sessions 

In [26]:
def distill_sequence(rs):
    seen = set(['Main_Page'])
    pages = []
    for r in rs:
        p = r['p'] 
        if p not in seen:
            pages.append(p)
            seen.add(p)
    return pages 

In [27]:
def filter_colon(ps):
    return [p for p in ps if ':' not in p]   

In [28]:
#requests.unpersist()

In [29]:
requests  = sc.textFile(input_dir) \
.map(lambda x: json.loads(x)['requests']) \
.map(parse_dates) \
.filter(lambda x: len(x) > 1) \
.flatMap(sessionize) \
.map(distill_sequence) \
.map(filter_colon) \
.filter(lambda x: len(x) > 1) \
.filter(lambda x: len(x) < 30) 

#requests.persist()
#requests.count()

In [30]:
save_rdd(requests.map(lambda x: '\t'.join(x)), '/home/ellery/readers/data' , '/user/ellery/readers/data', 'desktop_sessions')

256
256
256
256
0
0
0
0
